# Overview

Imagen on Vertex AI (image Generative AI) offers a variety of features:

* Image generation
* Image editing
* Visual captioning
* Visual question answering

This notebook focuses on **image generation** only.



Objectives
In this notebook, you will learn how to use the Vertex AI Python SDK to:

* Make request to https://fakestoreapi.com/products to retrieve 20 titles and descriptions
* Use title to generate an image
* Experiment with different parameters, such as:
  * number of images to be generated
  

Resources:
* https://cloud.google.com/vertex-ai/generative-ai/docs/image/overview
* https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/creating_high_quality_visual_assets_with_gemini_and_imagen.ipynb

**Costs**
* This notebook uses billable components of Google Cloud:
  * Vertex AI (Imagen)
* Learn about Vertex AI pricing and use the Pricing Calculator to generate a cost estimate based on your projected usage.

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
# Define project information
from google.colab import userdata
PROJECT_ID = userdata.get('VERTEX_AI_PROJECT_ID')
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

- Make request to https://fakestoreapi.com/products to retrieve title and description

In [ ]:
import requests

response = requests.get("https://fakestoreapi.com/products")
apiData = response.json()
products = []
for product in apiData:
    products.append({"title": product["title"], "description": product["description"]})

products

## Generate the first image

The model names from Vertex AI Imagen have two components: model name and version number. The naming convention follow this format: <model-name>@<version-number>. For example, imagegeneration@006 represent the version 006 of the **imagegeneration** model.

In [ ]:
from vertexai.preview.vision_models import ImageTextModel, ImageGenerationModel

model_name = "imagegeneration@006"
model = ImageGenerationModel.from_pretrained(model_name)


In [ ]:
import math
import matplotlib.pyplot as plt


# An auxiliary function to display images in grid
def display_images_in_grid(images):
    nrows = math.ceil(len(images) / 4)  # Display at most 4 images per row
    ncols = min(len(images) + 1, 4)  # Adjust columns based on the number of images

    # Create a figure and axes for the grid layout.
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i]._pil_image)
            ax.set_aspect("equal")
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
title = products[0].get("title")
print(title)

response = model.generate_images(title, number_of_images=1)
display_images_in_grid(response.images)

Use description to generate image


In [ ]:
description = products[0].get("description")
print(description)

response = model.generate_images(description, number_of_images=1)
display_images_in_grid(response.images)

* Write a function to generate the specified number of images
- prompts
- number of images

In [34]:
def generate_images(prompts, number_of_images):
    images = []
    for prompt in prompts:
      response = model.generate_images(prompt, number_of_images=number_of_images)
      for image in response.images:
        images.append(image)

    return images

In [ ]:
index = 16
title = products[index].get("title")
description = products[index].get("description")
images = generate_images([title, description], number_of_images=2)

print (len(images))
print ("Title: ", title, "\nDescription", description)
display_images_in_grid(images)